In [583]:
from statsmodels.regression.rolling import RollingOLS
import pandas_datareader.data as web
import matplotlib.pyplot as plt
import statsmodels.api as sm
import pandas as pd
import numpy as np
import datetime as dt
import yfinance as yf
import pandas_ta # for technical analysis
from datetime import datetime, timedelta
from sklearn.preprocessing import StandardScaler


In [584]:
def get_index_components_from_html(link:str, start_date:str, end_date:str) -> pd.DataFrame:
    """Get all components of a market index

    Args:
        link (str): link that have all components of market index
        start_date (str): Download start date string (YYYY-MM-DD)
        end_date (str): Download end date string (YYYY-MM-DD)

    Returns:
        pd.DataFrame: dataframe with information of each component
    """
    components = pd.read_html(link)[4]
    tickers = components["Ticker"].unique().tolist()
    data = yf.download(tickers,start=start_date, end=end_date).stack()
    data.index.names = ["Date", "Ticker"]
    return data

In [585]:
link = "https://en.wikipedia.org/wiki/Nasdaq-100"
end_date = datetime.strptime('2023-12-31', "%Y-%m-%d")
start_date = end_date - timedelta(days=365*5)
data = get_index_components_from_html(link,start_date, end_date)

[*********************100%%**********************]  101 of 101 completed


In [586]:
original_colums = data.columns

## Calculate technical indicators

### 1. Garman-Klass Volatility
medida utilizada para evaliar la volatividad de un activo financiero. Si el valor numérico de la volatilidad Garman-Klass es del 15%, significa que, en promedio, se espera que el precio del activo fluctúe en un rango del 15% en relación con su precio medio durante el período de tiempo analizado.


In [587]:
data["Garman Klass"] = ((np.log(data["High"]) - np.log(data["Low"]))**2)/2 - (2*np.log(2)-1)*((np.log(data['Close'])-np.log(data['Open']))**2)

### 2. RSI - Relative Strength Index
Indicador de momentum que mide la magnitud de los movimientos de precios para determinar las condiciones de sobrecompra o sobreventa de un activo [0, 100].
Cuando el RSI está por encima de 70, se considera que el activo está sobrecomprado. Por otro lado, cuando el RSI está por debajo de 30, se considera que el activo está sobrevendido.



In [588]:
data["RSI"] = data.groupby(level="Ticker")['Close'].transform(lambda x: pandas_ta.rsi(close=x,length=25))

### 3. MACD - Moving Average Convergence Divergence
Indicador para identificar cambios en la fuerza, dirección, impulso y duración de una tendencia en el precio de un activo. Este indicador se compone de dos líneas principales: la línea MACD y la línea de señal. 
La Línea MACD es la diferencia entre dos medias móviles exponenciales (EMAs): una de 12 días y otra de 26 días.
La línea de señal es una EMA de 9 días de la línea MACD.

Interpretación numérica:

**Cruce de Líneas**: Los cruces entre la Línea MACD y la Línea de Señal pueden indicar posibles señales de compra o venta. Cuando la Línea MACD cruza por encima de la Línea de Señal, puede interpretarse como una señal alcista, mientras que un cruce por debajo puede interpretarse como una señal bajista.

**Histograma**: Los valores del histograma MACD, ya sea por encima o por debajo de cero, indican la fuerza y ​​la dirección de la tendencia. Valores positivos sugieren una tendencia alcista, mientras que valores negativos sugieren una tendencia bajista.


In [589]:
def compute_macd(close):
    macd = pandas_ta.macd(close=close, length=20).iloc[:,0]
    return macd.sub(macd.mean()).div(macd.std())


In [590]:
data['MACD'] = data.groupby(level="Ticker", group_keys=False)['Close'].apply(compute_macd)

### 4. Bollinger Bands
Estas bandas son un tipo de envelope que se colocan alrededor del precio de un activo y están formadas por tres líneas:

**Línea Central (Media Móvil Simple)**: La línea central es una media móvil simple (SMA) que generalmente se calcula utilizando el precio de cierre durante un período específico. El valor predeterminado suele ser una SMA de 20 periodos.

**Banda Superior (Upper Band)**: Esta banda se encuentra por encima de la línea central y generalmente se calcula sumando dos desviaciones estándar al precio de la SMA.

**Banda Inferior (Lower Band)**: Esta banda se encuentra por debajo de la línea central y se calcula restando dos desviaciones estándar al precio de la SMA.

Interpretación de las Bandas de Bollinger:

**Rangos de Precios Esperados**: Las bandas superiores e inferiores actúan como niveles de resistencia y soporte dinámicos. 

**Volatilidad**: Cuando las bandas están más separadas, indica mayor volatilidad en el mercado.

**Cruces**: Los cruces de precios de un extremo a otro de las bandas pueden indicar posibles cambios de tendencia. Por ejemplo, si el precio cruza la banda superior, podría sugerir condiciones de sobrecompra, mientras que un cruce por debajo de la banda inferior podría indicar condiciones de sobreventa.

**Divergencias**: Divergencias entre el precio y las Bandas de Bollinger pueden ser señales de posibles cambios en la dirección del precio.

In [591]:
# We need to normalize close data
data['BB Low'] = data.groupby(level="Ticker")['Close'].transform(lambda x: pandas_ta.bbands(close=np.log1p(x), length=25).iloc[:,0])
                                                          
data['BB Mid'] = data.groupby(level="Ticker")['Close'].transform(lambda x: pandas_ta.bbands(close=np.log1p(x), length=25).iloc[:,0])
                                                          
data['BB High'] = data.groupby(level="Ticker")['Close'].transform(lambda x: pandas_ta.bbands(close=np.log1p(x), length=25).iloc[:,0])

### 5. ATR - Average True Range
Medir la volatilidad de un activo financiero.

Mayor ATR: Indica mayor volatilidad en el mercado, lo que puede ser útil para operadores que buscan aprovechar movimientos significativos de precios.

Menor ATR: Indica menor volatilidad, lo que podría ser de interés para aquellos que prefieren mercados más estables y predecibles.

In [592]:
def compute_atr(stock_data):
    atr = pandas_ta.atr(high=stock_data['High'],
                        low=stock_data['Low'],
                        close=stock_data['Close'],
                        length=14)
    
    return  atr.sub(atr.mean()).div(atr.std())

In [593]:
data['ATR'] = data.groupby(level="Ticker", group_keys=False).apply(compute_atr)

### 6. Dollar volumen - Average True Range
Es una medida que combina el número de acciones o contratos negociados con el precio de esos activos en un periodo de tiempo específico.

In [594]:
data['Dollar Volume'] = (data['Close']*data['Volume'])/1e6

Filter most liquid stocks by dollar volume.

In [595]:
# We will focus on a monthly level indicator

# Compute the average of the dolar volumen over each month
dolar_volume_monthly = data.unstack("Ticker")["Dollar Volume"].resample("M").mean().stack("Ticker").to_frame("Dollar Volume")

# Select just indicator columns
indicators = list(set(data.columns) - set(original_colums) - set(["Dollar Volume"]))
indicators.append("Close")
indicators_montly = data.unstack()[indicators].resample("M").last().stack("Ticker")

new_data = pd.concat([dolar_volume_monthly, indicators_montly], axis=1).dropna()


In [596]:
# Filter based on the year rolling avg of dolalr volume
new_data["Dollar Volume"] = new_data["Dollar Volume"].unstack("Ticker").rolling(12).mean().stack()
new_data["Dollar Volumen Rank"] = (new_data.groupby('Date')['Dollar Volume'].rank(ascending=False))

In [598]:
# Get the best 50
new_data = new_data[new_data["Dollar Volumen Rank"]<50].drop(['Dollar Volume', 'Dollar Volumen Rank'], axis=1) 

In [603]:
a = data.groupby(level=1, group_keys=False).get_group("AAPL")
a["Close"]


Date        Ticker
2019-01-02  AAPL       39.480000
2019-01-03  AAPL       35.547501
2019-01-04  AAPL       37.064999
2019-01-07  AAPL       36.982498
2019-01-08  AAPL       37.687500
                         ...    
2023-12-22  AAPL      193.600006
2023-12-26  AAPL      193.050003
2023-12-27  AAPL      193.149994
2023-12-28  AAPL      193.580002
2023-12-29  AAPL      192.529999
Name: Close, Length: 1258, dtype: float64

In [606]:
a['Close'].pct_change(1)

Date        Ticker
2019-01-02  AAPL           NaN
2019-01-03  AAPL     -0.099607
2019-01-04  AAPL      0.042689
2019-01-07  AAPL     -0.002226
2019-01-08  AAPL      0.019063
                        ...   
2023-12-22  AAPL     -0.005547
2023-12-26  AAPL     -0.002841
2023-12-27  AAPL      0.000518
2023-12-28  AAPL      0.002226
2023-12-29  AAPL     -0.005424
Name: Close, Length: 1258, dtype: float64

In [605]:
a['Close'].pct_change(1).pipe(lambda x: x.clip(lower=x.quantile(0.005),upper=x.quantile(1-0.005)))

Date        Ticker
2019-01-02  AAPL           NaN
2019-01-03  AAPL     -0.064841
2019-01-04  AAPL      0.042689
2019-01-07  AAPL     -0.002226
2019-01-08  AAPL      0.019063
                        ...   
2023-12-22  AAPL     -0.005547
2023-12-26  AAPL     -0.002841
2023-12-27  AAPL      0.000518
2023-12-28  AAPL      0.002226
2023-12-29  AAPL     -0.005424
Name: Close, Length: 1258, dtype: float64

In [ ]:
# Calculate retuns
def calculate_returns(df):
    """Calculate returns over various time lags and handle outliers.

    Args:
        df (DataFrame): Input DataFrame containing financial data.

    Returns:
        DataFrame: Original DataFrame with additional columns for returns calculated over specified time lags.
    """
    outlier_cutoff = 0.005

    lags = [1, 2, 3, 6, 9, 12]

    for lag in lags:

        df[f'return_{lag}m'] = (df['adj close']
                              .pct_change(lag)
                              .pipe(lambda x: x.clip(lower=x.quantile(outlier_cutoff),
                                                     upper=x.quantile(1-outlier_cutoff)))
                              .add(1)
                              .pow(1/lag)
                              .sub(1))
    return df
    
    
data = data.groupby(level=1, group_keys=False).apply(calculate_returns).dropna()

data
